<h1>Section1:Scrapping wiki link

<h4> Importing Libraries

In [1]:
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup as bs
import requests
import folium

<h4>using beautiful soup to scrap webpage

In [2]:
endpoint = r"https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url=endpoint).text
soup = bs(res,"lxml")

<h4>figure out the header of the table 

In [3]:
fsa_table = soup.find("table",attrs={"class":"wikitable"})
fsa_table_data =  fsa_table.tbody.find_all("tr")
fsa_table_head = fsa_table_data[0]
#fsa_table_data = fsa_table_data[1:]

<h4> Create the list with table data

In [4]:
fsa_table_data =  fsa_table.tbody.find_all("tr")
fsa_table_data

t_headers=[]
for th in fsa_table.tbody.find_all("th"):
    t_headers.append(th.text.replace("\n","").strip())

t_data=[]
 
for tr in  fsa_table.tbody.find_all("tr"):
    row = {}
    for td,th in zip(tr.find_all("td"),t_headers):
        row[th]=td.text.replace("\n","").strip()
    t_data.append(row)
t_data = t_data[1:]



<h4> Create final ilst 

In [5]:
t_data_final=[]
for items in t_data:
    if items["Borough"]!='Not assigned':
        items["Neighborhood"]=items["Neighborhood"].replace("/",",") #Combine multiple neighbourhoods 
        if items["Neighborhood"]=="":
            items["Neighborhood"]= items["Borough"] #If Borough is assigned and neighbourhood is not then neighbourhood is same as borough
        t_data_final.append(items)
t_data_final

[{'Postal code': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'},
 {'Postal code': 'M4A',
  'Borough': 'North York',
  'Neighborhood': 'Victoria Village'},
 {'Postal code': 'M5A',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Regent Park , Harbourfront'},
 {'Postal code': 'M6A',
  'Borough': 'North York',
  'Neighborhood': 'Lawrence Manor , Lawrence Heights'},
 {'Postal code': 'M7A',
  'Borough': 'Downtown Toronto',
  'Neighborhood': "Queen's Park , Ontario Provincial Government"},
 {'Postal code': 'M9A',
  'Borough': 'Etobicoke',
  'Neighborhood': 'Islington Avenue'},
 {'Postal code': 'M1B',
  'Borough': 'Scarborough',
  'Neighborhood': 'Malvern , Rouge'},
 {'Postal code': 'M3B', 'Borough': 'North York', 'Neighborhood': 'Don Mills'},
 {'Postal code': 'M4B',
  'Borough': 'East York',
  'Neighborhood': 'Parkview Hill , Woodbine Gardens'},
 {'Postal code': 'M5B',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Garden District, Ryerson'},
 {'Postal code': 'M6B', 'Bo

<h4> Convert to the pandas dataframe

In [6]:
df1 = pd.DataFrame(t_data_final)
df1.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


<h4>print shape of the frame - it has 103 rows and 3 cols

In [7]:
df1.shape

(103, 3)

<h1> End of section 1

<h1> Section2 : Adding Lat / Long to the postal Code

<h4>calling endpoint and exploring the data

In [8]:
endpoint = r"http://cocl.us/Geospatial_data"
res = requests.get(url=endpoint).text
res

'Postal Code,Latitude,Longitude\r\nM1B,43.8066863,-79.1943534\r\nM1C,43.7845351,-79.1604971\r\nM1E,43.7635726,-79.1887115\r\nM1G,43.7709921,-79.2169174\r\nM1H,43.773136,-79.2394761\r\nM1J,43.7447342,-79.2394761\r\nM1K,43.7279292,-79.2620294\r\nM1L,43.7111117,-79.2845772\r\nM1M,43.716316,-79.2394761\r\nM1N,43.692657,-79.2648481\r\nM1P,43.7574096,-79.273304\r\nM1R,43.7500715,-79.2958491\r\nM1S,43.7942003,-79.2620294\r\nM1T,43.7816375,-79.3043021\r\nM1V,43.8152522,-79.2845772\r\nM1W,43.7995252,-79.3183887\r\nM1X,43.8361247,-79.2056361\r\nM2H,43.8037622,-79.3634517\r\nM2J,43.7785175,-79.3465557\r\nM2K,43.7869473,-79.385975\r\nM2L,43.7574902,-79.3747141\r\nM2M,43.789053,-79.4084928\r\nM2N,43.7701199,-79.4084928\r\nM2P,43.7527583,-79.4000493\r\nM2R,43.7827364,-79.4422593\r\nM3A,43.7532586,-79.3296565\r\nM3B,43.7459058,-79.352188\r\nM3C,43.7258997,-79.340923\r\nM3H,43.7543283,-79.4422593\r\nM3J,43.7679803,-79.4872619\r\nM3K,43.7374732,-79.4647633\r\nM3L,43.7390146,-79.5069436\r\nM3M,43.728496

In [9]:
lat_long=[]
for str1 in res.split("\r"):
    lat_long_temp=[]
    for str2 in str1.split(","):
        lat_long_temp.append(str2.replace("\n","").strip())
    lat_long.append(lat_long_temp)

In [10]:
header = lat_long[0]
data = lat_long[1:]

In [11]:
header

['Postal Code', 'Latitude', 'Longitude']

<h4> Converting data to pandas dataframe

In [12]:
columns=["Postal code","lat","long"]
df2 = pd.DataFrame(data=data,columns=columns)
df2.head()

,Postal code,lat,long
0,M1B,43.8066863,-79.1943534
1,M1C,43.7845351,-79.1604971
2,M1E,43.7635726,-79.1887115
3,M1G,43.7709921,-79.2169174
4,M1H,43.773136,-79.2394761


<h4> Merging the two dataframe

In [13]:
df3 = pd.merge(df1,df2,on=["Postal code"],how="inner")

<h4>Final Dataframe

In [14]:
df3.head()

,Postal code,Borough,Neighborhood,lat,long
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.4647633
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6623015,-79.3894938


<h1>End of Section 2

<h1> Section3:Culstering and mapping 

<h4> Filter Borough having "Toronto"

In [15]:
list1=[]
for rows,cols in df3.iterrows():
    row_val={}
    if "Toronto" in cols[1]:
        row_val["Postal code"]=cols[0]
        row_val["Borough"]=cols[1]
        row_val["Neighborhood"]=cols[2]
        row_val["lat"]=cols[3]
        row_val["long"]=cols[4]
        list1.append(row_val)
df4 = pd.DataFrame(list1)
df4["lat"]=df4["lat"].astype(float)
df4["long"]=df4["long"].astype(float)
df4.head()

,Postal code,Borough,Neighborhood,lat,long
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


<h4> Reset the cols to bring neighbourhood to the first col 

In [17]:
df4.insert(loc=0,column="hood",value=df4["Neighborhood"])
df4.drop("Neighborhood",axis=1,inplace=True)
df4.rename(columns={"hood":"Neighborhood"},inplace=True)
df4.head()

,Neighborhood,Postal code,Borough,lat,long
0,"Regent Park , Harbourfront",M5A,Downtown Toronto,43.654260,-79.360636
1,"Queen's Park , Ontario Provincial Government",M7A,Downtown Toronto,43.662301,-79.389494
2,"Garden District, Ryerson",M5B,Downtown Toronto,43.657162,-79.378937
3,St. James Town,M5C,Downtown Toronto,43.651494,-79.375418
4,The Beaches,M4E,East Toronto,43.676357,-79.293031


<h4> plot using folium - This is without clustering 

In [18]:
toronto_map = folium.Map(location=[df4.loc[0,"lat"],df4.loc[0,"long"]],zoom_start=10)

for lat , long , label,Borough in zip(df4["lat"],df4["long"],df4["Postal code"],df4["Borough"]):
    label = "Borough: " + Borough + ", Postal code:" + label
    folium.features.CircleMarker(
        [lat, long],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color = 'yellow',
        fill_opacity = 0.6
    ).add_to(toronto_map)
toronto_map

<h4> Prep for clustering on neighbourhood - unpack the Neighborhood col

In [19]:
list2=[]
for rows,cols in df4.iterrows():
    for items in cols[2].split(","):
        row_val={}
        row_val["Postal code"]=cols[0]
        row_val["Borough"]=cols[1]
        row_val["Neighborhood"]=items
        row_val["lat"]=cols[3]
        row_val["long"]=cols[4]
        list2.append(row_val)
df5 = pd.DataFrame(list2)
df5.head()

,Postal code,Borough,Neighborhood,lat,long
0,"Regent Park , Harbourfront",M5A,Downtown Toronto,43.654260,-79.360636
1,"Queen's Park , Ontario Provincial Government",M7A,Downtown Toronto,43.662301,-79.389494
2,"Garden District, Ryerson",M5B,Downtown Toronto,43.657162,-79.378937
3,St. James Town,M5C,Downtown Toronto,43.651494,-79.375418
4,The Beaches,M4E,East Toronto,43.676357,-79.293031


In [20]:
df5.insert(loc=0,column="hood",value=df5["Neighborhood"])
df5.drop("Neighborhood",axis=1,inplace=True)
df5.rename(columns={"hood":"Neighborhood"},inplace=True)

In [21]:
df5.head()
df5["lat"]=df5["lat"].astype("float")
df5["long"]=df5["long"].astype("float")
df5.head()

,Neighborhood,Postal code,Borough,lat,long
0,Downtown Toronto,"Regent Park , Harbourfront",M5A,43.654260,-79.360636
1,Downtown Toronto,"Queen's Park , Ontario Provincial Government",M7A,43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937
3,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418
4,East Toronto,The Beaches,M4E,43.676357,-79.293031


<h4>One hot encoding based on neighborhood

In [22]:
toronto_onehot = pd.get_dummies(df5[['Neighborhood']], prefix="", prefix_sep="")

In [23]:
toronto_onehot.insert(loc=0,column="Postal code",value=df5["Postal code"])
toronto_onehot.insert(loc=1,column="lat",value=df5["lat"])
toronto_onehot.insert(loc=1,column="long",value=df5["long"])
toronto_onehot

,Postal code,long,lat,Central Toronto,Downtown Toronto,East Toronto,West Toronto
0,"Regent Park , Harbourfront",-79.360636,43.654260,0,1,0,0
1,"Queen's Park , Ontario Provincial Government",-79.389494,43.662301,0,1,0,0
2,"Garden District, Ryerson",-79.378937,43.657162,0,1,0,0
3,St. James Town,-79.375418,43.651494,0,1,0,0
4,The Beaches,-79.293031,43.676357,0,0,1,0
5,Berczy Park,-79.373306,43.644771,0,1,0,0
6,Central Bay Street,-79.387383,43.657952,0,1,0,0
7,Christie,-79.422564,43.669542,0,1,0,0
8,"Richmond , Adelaide , King",-79.384568,43.650571,0,1,0,0
9,"Dufferin , Dovercourt Village",-79.442259,43.669005,0,0,0,1


In [24]:
toronto_groupby = toronto_onehot.groupby(["Postal code","lat","long"]).mean().reset_index()

In [25]:
toronto_groupby.shape

(39, 7)

<h4> start culstering

In [26]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=4,random_state=0)
X=toronto_groupby.drop(["Postal code","lat","long"],axis=1)
km.fit(X)
labels=km.labels_
center = km.cluster_centers_
km.labels_[0:75] 

array([0, 2, 3, 0, 0, 0, 0, 0, 1, 1, 2, 0, 1, 0, 0, 2, 3, 0, 1, 2, 1, 1,
       2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 3, 1, 1, 3, 3, 0, 0])

<h4> Add cluster groupings to the main dataframe

In [27]:
toronto_groupby.insert(loc=1,value=km.labels_,column="cluster_label")
toronto_groupby.head()

,Postal code,cluster_label,lat,long,Central Toronto,Downtown Toronto,East Toronto,West Toronto
0,Berczy Park,0,43.644771,-79.373306,0,1,0,0
1,"Brockton , Parkdale Village , Exhibition Place",2,43.636847,-79.428191,0,0,0,1
2,Business reply mail Processing CentrE,3,43.662744,-79.321558,0,0,1,0
3,"CN Tower , King and Spadina , Railway Lands , ...",0,43.628947,-79.394420,0,1,0,0
4,Central Bay Street,0,43.657952,-79.387383,0,1,0,0


<h4> Plot the map , this time using clustering

In [28]:
def return_color(value):
    if value==0:
        return("red")
    elif value==1:
        return("blue")
    elif value==2:
        return("yellow")
    else:
        return("black")

In [29]:
toronto_groupbymap= folium.Map(location=[toronto_groupby.loc[0,"lat"],toronto_groupby.loc[0,"long"]],zoom_start=10)

for lat , long , pcode,clusterlabel in zip(toronto_groupby["lat"],toronto_groupby["long"],toronto_groupby["Postal code"],toronto_groupby["cluster_label"]):
    label = "cluster: " + str(clusterlabel) #+ ", Postal code:" + str(pcode)
    folium.features.CircleMarker(
        [lat, long],
        radius=5,
        color=return_color(clusterlabel),
        popup=label,
        fill = True,
        fill_color = return_color(clusterlabel),
        fill_opacity = 0.6
    ).add_to(toronto_groupbymap)
toronto_groupbymap

<h1> End of section 3